In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model

np.random.seed(23)

Using TensorFlow backend.


Write a program to output the numbers 1-100, where numbers divisible by 3 output Fizz, divisible by 5 output Buzz, and divisible by both output FizzBuzz

In [2]:
# specifiy number of binary digits
num_digits = 12
# create training data not including 1-100, 
# since that is what to predict
raw_train = np.random.randint(101, 2**num_digits, 
                                  2**num_digits-101)
raw_train[:3]

array([ 696,  843, 1165])

In [19]:
def fizzbuzz(x):
    if x % 15== 0:
        return 'fizzbuzz'
    elif x % 3 == 0:
        return 'fizz'
    elif x % 5 == 0:
        return 'buzz'
    else:
        return x

In [4]:
# encode FizzBuzz
def encode_fizzbuzz(x):
    '''
    1,0,0,0 = fizzbuzz
    0,1,0,0 = fizz
    0,0,1,0 = buzz
    0,0,0,1 = original number
    '''
    if x % 15== 0:
        return np.array([0,0,0,1])
    elif x % 3 == 0:
        return np.array([0,1,0,0])
    elif x % 5 == 0:
        return np.array([0,0,1,0])
    else:
        return np.array([1,0,0,0])

In [5]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

In [6]:
encoded_train = [binary_encode(i, num_digits) for i in raw_train]

In [7]:
x_train = np.array(encoded_train)
y_train = np.array([encode_fizzbuzz(i) for i in raw_train])

In [8]:
len(x_train)

3995

In [9]:
len(y_train)

3995

In [10]:
# decode FizzBuzz, requires second input for original number
def decode_fizzbuzz(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

In [30]:
# Now let's build our model, add layers, compile, and fit it!
model = Sequential()

## Add the model's layers.
# These are defaults and they work pretty well
model.add(Dense(1000, input_dim=num_digits, activation="relu"))
model.add(Dense(1000, activation="relu"))
model.add(Dense(4, activation="softmax"))

model.compile(loss='categorical_crossentropy', 
                  optimizer='adagrad', metrics=["accuracy"])

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 1000)              13000     
_________________________________________________________________
dense_5 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 4004      
Total params: 1,018,004
Trainable params: 1,018,004
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(x_train, y_train, nb_epoch=30, batch_size=128)

Epoch 1/30
3995/3995 [==============================] - 2s 477us/step - loss: 1.3321 - accuracy: 0.5011
Epoch 2/30
3995/3995 [==============================] - 1s 354us/step - loss: 1.1447 - accuracy: 0.5244
Epoch 3/30
3995/3995 [==============================] - 1s 302us/step - loss: 1.1316 - accuracy: 0.5267
Epoch 4/30
3995/3995 [==============================] - 1s 339us/step - loss: 1.1136 - accuracy: 0.5287
Epoch 5/30
3995/3995 [==============================] - 1s 351us/step - loss: 1.0886 - accuracy: 0.5367
Epoch 6/30
3995/3995 [==============================] - 1s 343us/step - loss: 1.0570 - accuracy: 0.5452
Epoch 7/30
3995/3995 [==============================] - 1s 310us/step - loss: 1.0162 - accuracy: 0.5537
Epoch 8/30
3995/3995 [==============================] - 1s 306us/step - loss: 0.9626 - accuracy: 0.5770
Epoch 9/30
3995/3995 [==============================] - 1s 333us/step - loss: 0.8985 - accuracy: 0.6125
Epoch 10/30
3995/3995 [==============================] - 1s 332u

In [33]:
numbers = np.arange(1,101)
x_test = np.transpose(binary_encode(numbers, num_digits))
x_test[:2]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [34]:
y_test = model.predict_classes(x_test)

In [35]:
predictions = np.vectorize(decode_fizzbuzz)(numbers, y_test)

In [47]:
print(predictions)

['1' '2' '3' '4' '5' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13' '14'
 'fizzbuzz' '16' '17' '18' '19' '20' '21' '22' '23' 'fizz' '25' '26'
 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz' 'fizz' '37' '38'
 '39' 'buzz' '41' 'fizz' '43' '44' 'fizzbuzz' 'fizz' '47' '48' '49' 'buzz'
 'fizz' '52' '53' '54' '55' '56' 'fizz' '58' '59' 'fizzbuzz' '61' '62'
 'fizz' '64' '65' 'fizz' '67' '68' '69' 'buzz' '71' 'fizz' '73' '74'
 'fizzbuzz' '76' '77' 'fizz' '79' '80' 'fizz' '82' '83' 'fizz' '85' '86'
 'fizz' '88' '89' 'fizzbuzz' '91' '92' 'fizz' '94' 'buzz' '96' '97' '98'
 '99' 'buzz']


In [41]:
answer = [fizzbuzz(i) for i in range(1,101)]

In [42]:
#evaluate
evaluate = np.array(answer == predictions)
print(np.count_nonzero(evaluate == True) / 100)

0.83
